<a href="https://colab.research.google.com/github/pankajsingh09/Hands-on-Big-Data-Practices-with-PySpark-Spark-Tuning/blob/main/Intro_to_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 50 kB/s 
     |████████████████████████████████| 199 kB 78.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=722b55db6e70aacb25a309bac196e87d25a67e14d6a2824af4f6d9ca40759074
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark import SparkContext, SparkConf

In [4]:
#initializing Spark
conf = SparkConf().setAppName("testApp1").setMaster("local[*]") 

In [5]:
sc = SparkContext(conf=conf)

In [6]:
print(sc)

<SparkContext master=local[*] appName=testApp1>


In [7]:
sc.defaultParallelism

2

In [8]:
#Generate Random Number
import random
randomlist = random.sample(range(1,40),10)
print(randomlist)

[20, 4, 14, 9, 26, 37, 8, 31, 18, 11]


In [9]:
#create RDD
rdd1 = sc.parallelize(randomlist,4)
rdd1.collect()

[20, 4, 14, 9, 26, 37, 8, 31, 18, 11]

In [10]:
#Data Distribution in Partition
print(rdd1.getNumPartitions())
print(rdd1.glom().collect())
print("The first 2 partitions are :",rdd1.glom().take(2))

4
[[20, 4], [14, 9], [26, 37], [8, 31, 18, 11]]
The first 2 partitions are : [[20, 4], [14, 9]]


In [11]:
#print Last Partition
print(rdd1.glom().collect()[3])

[8, 31, 18, 11]


In [12]:
#count
rdd1.count()

10

In [13]:
#First
rdd1.first()

20

In [14]:
#Top
rdd1.top(2)

[37, 31]

In [15]:
#distinct
rdd1.distinct().collect()

[20, 4, 8, 9, 37, 14, 26, 18, 31, 11]

In [16]:
# Map
def myFunc(item):
  return (item + 1)*3

rdd_map = rdd1.map(myFunc)

rdd_map.glom().collect()

[[63, 15], [45, 30], [81, 114], [27, 96, 57, 36]]

In [17]:
# Map with Lambda Function

rdd_map = rdd1.map(lambda x : (x + 1 ) * 3)
rdd_map.glom().collect()

[[63, 15], [45, 30], [81, 114], [27, 96, 57, 36]]

In [18]:
# Filter
rdd_filter = rdd1.filter(lambda x : x%3==0)
print(rdd1.glom().collect())
rdd_filter.glom().collect()


[[20, 4], [14, 9], [26, 37], [8, 31, 18, 11]]


[[], [9], [], [18]]

In [19]:
# FlatMap

rdd_flatMap = rdd1.flatMap(lambda x : [x+2, x+5])
print(rdd_flatMap.collect())

# Reduce 
# Reduce do aggregation (add,sub,mul,etc)
rdd_flatMap.reduce(lambda x,y : x+y)
print(rdd_flatMap.collect())


[22, 25, 6, 9, 16, 19, 11, 14, 28, 31, 39, 42, 10, 13, 33, 36, 20, 23, 13, 16]


426

In [20]:
# Descriptive Statistics
print([rdd1.max(), rdd1.min(), rdd1.mean(), round(rdd1.stdev(),2), rdd1.sum()])

[37, 4, 17.8, 10.2, 178]


In [21]:
# mapPartition

def myfunct(x):
  sum = 0
  for item in x:
    sum = sum + item
  
  yield sum
# yield is similar to sum  but insted of returing a single value it return a list

rdd1.mapPartitions(myfunct).collect()

[24, 23, 63, 68]